# Quantum Computing for Molecular Systems

This notebook introduces quantum computing concepts for molecular modeling and drug discovery.

## Learning Objectives
- Understand basic quantum computing concepts
- Learn to use Qiskit and PennyLane
- Explore quantum algorithms for molecular problems
- Implement variational quantum eigensolvers (VQE)

In [ ]:
# Import quantum computing libraries
import numpy as np
import matplotlib.pyplot as plt

# Qiskit imports
from qiskit import QuantumCircuit, transpile
from qiskit.primitives import Estimator
from qiskit.quantum_info import SparsePauliOp
from qiskit_aer import AerSimulator

# PennyLane imports
import pennylane as qml
from pennylane import numpy as pnp

print("Quantum computing libraries loaded successfully!")

## 1. Basic Quantum Circuit with Qiskit

In [ ]:
# Create a simple quantum circuit
def create_bell_state():
    """
    Create a Bell state (quantum entangled state)
    """
    # Create a quantum circuit with 2 qubits and 2 classical bits
    qc = QuantumCircuit(2, 2)
    
    # Apply Hadamard gate to first qubit
    qc.h(0)
    
    # Apply CNOT gate
    qc.cx(0, 1)
    
    # Measure both qubits
    qc.measure([0, 1], [0, 1])
    
    return qc

# Create and visualize the circuit
bell_circuit = create_bell_state()
print("Bell State Circuit:")
print(bell_circuit.draw())

# Simulate the circuit
simulator = AerSimulator()
compiled_circuit = transpile(bell_circuit, simulator)
job = simulator.run(compiled_circuit, shots=1000)
result = job.result()
counts = result.get_counts()

print("\nMeasurement Results:")
for outcome, count in counts.items():
    print(f"State |{outcome}>: {count} times ({count/1000:.1%})")

## 2. Molecular Hamiltonians

In [ ]:
# Define a simple molecular Hamiltonian (H2 molecule)
def h2_hamiltonian():
    """
    Create a simplified Hamiltonian for H2 molecule
    Using the STO-3G basis set approximation
    """
    # Coefficients for H2 in minimal basis
    # These are simplified values for demonstration
    coeffs = [-1.0523732, 0.39793742, -0.39793742, -0.01128010]
    
    # Pauli operators
    ops = ['II', 'IZ', 'ZI', 'ZZ']
    
    # Create the Hamiltonian
    hamiltonian = SparsePauliOp.from_list([(op, coeff) for op, coeff in zip(ops, coeffs)])
    
    return hamiltonian

h2_ham = h2_hamiltonian()
print("H2 Molecule Hamiltonian:")
print(h2_ham)
print(f"\nNumber of qubits required: {h2_ham.num_qubits}")

## 3. Variational Quantum Eigensolver (VQE) with PennyLane

In [ ]:
# Set up PennyLane device
dev = qml.device('default.qubit', wires=2)

# Define the ansatz (trial wavefunction)
@qml.qnode(dev)
def vqe_ansatz(params):
    """
    Simple VQE ansatz for 2 qubits
    """
    # Apply RY rotations
    qml.RY(params[0], wires=0)
    qml.RY(params[1], wires=1)
    
    # Apply entangling gate
    qml.CNOT(wires=[0, 1])
    
    # Apply more rotations
    qml.RY(params[2], wires=0)
    qml.RY(params[3], wires=1)
    
    return qml.state()

# Define cost function (expectation value of Hamiltonian)
@qml.qnode(dev)
def cost_function(params):
    """
    Calculate expectation value of H2 Hamiltonian
    """
    # Apply the ansatz
    qml.RY(params[0], wires=0)
    qml.RY(params[1], wires=1)
    qml.CNOT(wires=[0, 1])
    qml.RY(params[2], wires=0)
    qml.RY(params[3], wires=1)
    
    # Measure expectation values for each Pauli string
    h_ii = -1.0523732 * qml.Identity(0) @ qml.Identity(1)
    h_iz = 0.39793742 * qml.Identity(0) @ qml.PauliZ(1)
    h_zi = -0.39793742 * qml.PauliZ(0) @ qml.Identity(1)
    h_zz = -0.01128010 * qml.PauliZ(0) @ qml.PauliZ(1)
    
    return qml.expval(h_ii + h_iz + h_zi + h_zz)

# Initialize parameters
np.random.seed(42)
params = np.random.uniform(0, 2*np.pi, size=4)

print(f"Initial parameters: {params}")
print(f"Initial energy: {cost_function(params):.6f} Ha")

## 4. Optimization with Classical Optimizer

In [ ]:
# Optimize the VQE parameters
from scipy.optimize import minimize

# Track optimization progress
energy_history = []

def callback(params):
    energy = cost_function(params)
    energy_history.append(energy)
    if len(energy_history) % 10 == 0:
        print(f"Step {len(energy_history)}: Energy = {energy:.6f} Ha")

# Run optimization
print("Starting VQE optimization...")
result = minimize(
    cost_function,
    params,
    method='COBYLA',
    callback=callback,
    options={'maxiter': 100}
)

print(f"\nOptimization completed!")
print(f"Final energy: {result.fun:.6f} Ha")
print(f"Optimal parameters: {result.x}")

# Known exact ground state energy for H2 (reference)
exact_energy = -1.136189
print(f"Exact ground state energy: {exact_energy:.6f} Ha")
print(f"Error: {abs(result.fun - exact_energy):.6f} Ha")

## 5. Visualization of Optimization Progress

In [ ]:
# Plot optimization convergence
plt.figure(figsize=(10, 6))
plt.plot(energy_history, 'b-', linewidth=2, label='VQE Energy')
plt.axhline(y=exact_energy, color='r', linestyle='--', linewidth=2, label='Exact Ground State')
plt.xlabel('Optimization Step')
plt.ylabel('Energy (Hartree)')
plt.title('VQE Optimization for H₂ Molecule')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Calculate final error
final_error = abs(energy_history[-1] - exact_energy)
print(f"\nFinal optimization error: {final_error:.6f} Ha")
print(f"Final optimization error: {final_error * 27.211:.3f} eV")  # Convert to eV

## 6. Quantum Circuit Visualization

In [ ]:
# Create and visualize the optimized quantum circuit
optimal_params = result.x

@qml.qnode(dev)
def optimized_circuit(params):
    qml.RY(params[0], wires=0)
    qml.RY(params[1], wires=1)
    qml.CNOT(wires=[0, 1])
    qml.RY(params[2], wires=0)
    qml.RY(params[3], wires=1)
    return qml.state()

# Print circuit
print("Optimized VQE Circuit:")
print(qml.draw(optimized_circuit)(optimal_params))

# Get the final quantum state
final_state = optimized_circuit(optimal_params)
print(f"\nFinal quantum state amplitudes:")
for i, amp in enumerate(final_state):
    if abs(amp) > 1e-6:
        print(f"|{i:02b}>: {amp:.4f}")

## Exercise

Try the following:
1. Modify the ansatz to include more parameters
2. Implement VQE for other small molecules (LiH, BeH₂)
3. Compare different optimizers (SPSA, Adam, etc.)
4. Explore quantum machine learning for molecular property prediction

## Next Steps
- Learn about quantum approximate optimization algorithm (QAOA)
- Explore quantum machine learning for drug discovery
- Study quantum algorithms for molecular dynamics
- Investigate near-term quantum advantages in chemistry